# Projeto de Modelagem Estatística - 2º Bimestre
**Aluno:** Felipe de Freitas e Benjamin Suzuki
**Disciplina:** Modelagem Estatística
**Dataset:** Sleep in Mammals (Fonte: Kaggle/OpenML)

## 1. Introdução
O objetivo deste projeto é analisar padrões de sono em mamíferos. Realizaremos duas tarefas:
1. **Regressão:** Prever o tempo total de sono (`total_sleep`).
2. **Classificação:** Prever se o animal dorme muito ou pouco (`sleep_class`).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from pycaret.regression import setup as setup_reg, compare_models as compare_reg
from pycaret.classification import setup as setup_clf, compare_models as compare_clf, plot_model
from pycaret.regression import *

SEED = 42

In [ ]:
df = pd.read_csv('dataset_2191_sleep.csv')

In [ ]:
df.head()

## 1.1 Limpeza e Tratamento dos Dados

In [ ]:
df = df.replace('?', np.nan)
df = df.fillna(df.mean())

In [ ]:
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
df.isnull().sum()

## 2. Análise Exploratória (EDA)
Investigamos a distribuição dos dados e correlações. O tratamento de dados incluiu a remoção de caracteres inválidos ('?') e imputação de valores ausentes pela média.

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 8))
corr = df.corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Matriz de Correlação - O que afeta o sono?")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df['total_sleep'], kde=True, color='purple')
plt.title("Distribuição das Horas de Sono (Alvo da Regressão)")
plt.show()

## 3. Modelagem de Regressão
Implementamos regressão linear simples, múltipla e polinomial. Utilizamos `statsmodels` para análise de p-valores e multicolinearidade (VIF).

In [ ]:
target_reg = 'total_sleep'
X = df.drop(columns=[target_reg])
y = df[target_reg]
X_sm = sm.add_constant(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y, test_size=0.3, random_state=42)
modelo_ols = sm.OLS(y_train, X_train).fit()
y_pred = modelo_ols.predict(X_test)
print(modelo_ols.summary())

In [ ]:
print("\n--- Métricas de Desempenho (Teste) ---")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.4f}")
print(f"R2: {r2_score(y_test, y_pred):.4f}")

## 3.1. Modelagem de Classificação

Para atender aos requisitos de classificação do projeto, realizamos uma engenharia de atributos criando a variável alvo **`sleep_class`**:
* **Classe 0 (Short Sleepers):** Animais que dormem menos ou igual à mediana.
* **Classe 1 (Long Sleepers):** Animais que dormem mais que a mediana.

Implementamos dois modelos:
1. **Naive Bayes (GaussianNB):** Probabilístico, assume independência entre atributos.
2. **Regressão Logística:** Modelo linear para classificação binária.

A avaliação será feita via **Matriz de Confusão** e métricas como Acurácia e AUC.

In [ ]:
median_sleep = df['total_sleep'].median()
df['sleep_class'] = (df['total_sleep'] > median_sleep).astype(int)

In [ ]:
X_clf = df.drop(columns=['total_sleep', 'sleep_class'])
y_clf = df['sleep_class']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_clf, y_clf, test_size=0.3, random_state=42)

In [ ]:
# Modelo 1: Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train_c, y_train_c)
y_pred_nb = nb_model.predict(X_test_c)

In [ ]:
# Modelo 2: Regressão Logística
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_c, y_train_c)
y_pred_lr = lr_model.predict(X_test_c)

In [ ]:
print("--- Naive Bayes ---")
print(classification_report(y_test_c, y_pred_nb))
print("\n--- Regressão Logística ---")
print(classification_report(y_test_c, y_pred_lr))

In [ ]:
# Criar variáveis polinomiais (Grau 2)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train) # usa o X_train da regressão anterior
X_test_poly = poly.transform(X_test)

# Treinar modelo
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

y_pred_poly = poly_model.predict(X_test_poly)
r2_poly = r2_score(y_test, y_pred_poly)

print(f"R² Regressão Linear: {r2_score(y_test, y_pred):.4f}")
print(f"R² Regressão Polinomial: {r2_poly:.4f}")

if r2_poly > r2_score(y_test, y_pred):
    print("CONCLUSÃO: A relação não é linear. O modelo polinomial se ajustou melhor.")
else:
    print("CONCLUSÃO: O aumento da complexidade não trouxe ganho real.")

## 4. Otimização e Comparação
Utilizamos **PyCaret** para comparar múltiplos algoritmos (AutoML) e **GridSearch** (Sklearn) para refinar os hiperparâmetros da Regressão Logística.

In [ ]:
# Definindo o modelo e os parâmetros para testar
param_grid = {'C': [0.1, 1, 10, 100], 'solver': ['liblinear', 'lbfgs']}
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy')

# Treinando (usando seus dados de classificação X_train_c, y_train_c)
grid.fit(X_train_c, y_train_c)

print("Melhores parâmetros encontrados (Sklearn):", grid.best_params_)
print("Melhor Acurácia:", grid.best_score_)

In [ ]:
# Configuração do Experimento (Setup)
reg_exp = setup(data=df, target='total_sleep', session_id=42, verbose=False)

# Comparar Modelos
print("--- Comparando Modelos de Regressão ---")
best_reg_model = compare_models(n_select=1)

# Tuning (Otimização de Hiperparâmetros)
print("\n--- Otimizando o Melhor Modelo ---")
tuned_reg_model = tune_model(best_reg_model)

# Resultados Finais
print("\n--- Melhores Parâmetros Encontrados ---")
print(tuned_reg_model)

In [ ]:
df_clf_final = df.drop(columns=['total_sleep'])

print("Iniciando PyCaret...")
clf_exp = setup(data=df_clf_final, target='sleep_class', session_id=42, verbose=False)
best_model = compare_models(n_select=1)
print(f"\nMelhor modelo encontrado: {best_model}")

try:
    print("\nGerando Gráficos...")
    plot_model(best_model, plot='auc')
    plot_model(best_model, plot='confusion_matrix')
except Exception as e:
    print(f"Aviso visual: {e}")

## 5. Conclusão
Os resultados indicam que variáveis como **Danger Index** (Índice de Perigo) e **Body Weight** são os maiores preditores do sono.
* O modelo polinomial apresentou leve melhoria sobre o linear, sugerindo relações não-lineares.
* O PyCaret identificou que modelos baseados em árvore (Random Forest/Extra Trees) superaram a regressão linear básica.